In [111]:
import pandas as pd
import json
import ast

In [12]:
path_data_to_clean = "../Datas/Doccano/doccano_skills_outputs.jsonl"

### Fonctions permettant de clean un JSONL file

In [112]:
def read_jsonl(path_json_file):
    with open(path_json_file, 'r') as json_file:
        json_list = list(json_file)
    return json_list

def clean_string(doc):
    """Permet de clean un string"""
    doc = doc.replace('SAUTDELIGNE', '')
    return doc

def clean_full_json(path_json_file, path_new_file):
    """Permet de clean un fichier jsonl"""
    with open(path_json_file, 'r') as json_file:
        json_list = list(json_file)
    
    #On met le tout au format texte, sous la forme recquise par Doccano
    doc = ''
    for json_str in json_list:
        #d in a string of dict
        d = json_str
        doc += str(d)
    
    #On nettoie tout le texte
    doc = clean_string(doc)
    
    #On save au jsonl format pour Doccano (mettre .jsonl dans le path_new_file)
    with open(path_new_file, "w") as file:
        file.write(doc)

In [78]:
clean_full_json(path_data_to_clean, "../Datas/skills_doccano_inputs_new.jsonl")

### Doccano Pb exportation
###### Ce bout de code permet de mettre au bon format ce qui a ete exporté par Doccano

In [117]:
def oneDictToGoodFormat(python_dict, labels):
    """Parfois en exportant les CV labelises via Doccano, le format n'est pas bon.
    Cette fonction sert a mettre CHAQUE output au bon format"""
    old_labels = python_dict['annotations']
    new_labels = []
    for d in old_labels:
        try:
            sublist = []
            # Retrieve Start index, End index, Name label
            s = d['start_offset']
            e = d['end_offset']
            name = d['label']
            new_name = labels[str(name)]
            sublist.extend([s, e, new_name])
            new_labels.append(sublist)
        except:
            #Il s'agit du cas ou certains labels sont presents dans le json mais pas dans le dictionnaire "labels"
            pass
    # Drop useful element
    del python_dict['annotations']
    del python_dict['comment_count']
    python_dict["labels"] = new_labels
    return python_dict

In [114]:
def FullFileToGoodFormat(path_new_file, list_of_labelized_datas):
    """Permet de mettre le JSON complet issu de DOCCANO au bon 
    format pour RUN le NER"""
    with open(path_new_file, "w") as file:
        for i in range(len(list_of_labelized_datas)):
            python_dict = json.loads(list_of_labelized_datas[i])
            python_dict = oneDictToGoodFormat(python_dict, labels)
            file.write(json.dumps(python_dict))
            file.write('\n')

In [106]:
# Liste des labels définis dans DOCCANO
labels = {
    '1': "U-SKILLS", '2': "B-SKILLS", '3' :"I-SKILLS", '4':"L-SKILLS", '5':"U-ANGLAIS",'6':"U-NIVEAU_ANGLAIS",
    '7':"B-NIVEAU_ANGLAIS",'8':"I-NIVEAU_ANGLAIS", '9':"L-NIVEAU_ANGLAIS", '10':"U-OTHER_LANGUAGE",
    '11':"U-ATOUTS", '12':"B-ATOUTS", '13':"I-ATOUTS", '14':"L-ATOUTS"
  }

In [107]:
# Lecture du JSON issu de DOCCANO
path_data_to_clean = "../Datas/Doccano/doccano_skills_outputs.jsonl"
list_of_labelized_datas = read_jsonl(path_data_to_clean)

In [109]:
# Creation du nouiveau JSON au bon format
FullFileToGoodFormat("../Datas/Doccano/doccano_skills_outputs.jsonl", list_of_labelized_datas)

In [118]:
# Liste des labels définis dans DOCCANO SANS ATOUTS
labels = {
    '1': "U-SKILLS", '2': "B-SKILLS", '3' :"I-SKILLS", '4':"L-SKILLS", '5':"U-ANGLAIS",'6':"U-NIVEAU_ANGLAIS",
    '7':"B-NIVEAU_ANGLAIS",'8':"I-NIVEAU_ANGLAIS", '9':"L-NIVEAU_ANGLAIS", '10':"U-OTHER_LANGUAGE"
  }